In [2]:
from collections import defaultdict
from surprise import SVD
import numpy as np
import pandas as pd

In [9]:
# load data from all 6 datasets
phn1 = pd.read_csv('phone_user_review_file_1.csv',encoding='Latin-1')
phn2 = pd.read_csv('phone_user_review_file_2.csv',encoding='Latin-1')
phn3 = pd.read_csv('phone_user_review_file_3.csv',encoding='Latin-1')
phn4 = pd.read_csv('phone_user_review_file_4.csv',encoding='Latin-1')
phn5 = pd.read_csv('phone_user_review_file_5.csv',encoding='Latin-1')
phn6 = pd.read_csv('phone_user_review_file_6.csv',encoding='Latin-1')


In [13]:
# combine datasets to form 1 single dataset

(163837, 11)

In [14]:
phone_users = pd.concat([phn1,phn2,phn3,phn4,phn5,phn6], axis=0)

In [15]:
phone_users.shape

(1415133, 11)

In [16]:
phone_users.head(20)

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8
5,/cellphones/samsung-galaxy-s8/,5/10/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,I am the type of person who never would comple...,BDB76,Samsung Galaxy S8
6,/cellphones/samsung-galaxy-s8/,5/10/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,The way this Samsung S8 phone operates is more...,KLC30306,Samsung Galaxy S8
7,/cellphones/samsung-galaxy-s8/,5/10/2017,en,us,Verizon Wireless,verizonwireless.com,6.0,10.0,I bought this phone very excited to use it. I ...,Mnhy,Samsung Galaxy S8
8,/cellphones/samsung-galaxy-s8/,5/10/2017,en,us,Verizon Wireless,verizonwireless.com,6.0,10.0,It is an extremely advanced and truly a Smart ...,Mr Alan,Samsung Galaxy S8
9,/cellphones/samsung-galaxy-s8/,5/10/2017,en,us,Verizon Wireless,verizonwireless.com,8.0,10.0,"Great phone with a PHENOMENAL Camera, not all ...",Raven Wolf Song,Samsung Galaxy S8


Some valuable inforation can be obtained from above data
1. countrywise product distriution 
2. yearwise product distribution 
3. product and service provider correlation 
4. highest rated product

In [47]:
print(phone_users['country'].unique())
print(phone_users['product'].unique())
print(phone_users['source'].unique())
print(phone_users['score'].unique())

['us' 'gb' 'au' 'in' 'il' 'fi' 'it' 'no' 'nl' 'ru' 'cz' 'be' 'se' 'es'
 'br' 'fr' 'ua' 'ca' 'de' 'dk' 'nz' 'tr' 'pl' 'ch' 'pt' 'hu' 'za' 'mx'
 'ae' 'id' 've' 'sg' 'cl' 'ar' 'ec' 'tw' 'co' 'jp' 'uy' 'pe' 'cn' 'kr']
['Samsung Galaxy S8'
 'Samsung Galaxy S8 (64GB) G950U 5.8" 4G LTE Unlocked (GSM + CDMA, US WARRANTY) (Orchid Gray)'
 'Samsung Galaxy S8 64GB (AT&T)' ... 'Maxon MX-3204' 'Maxon 3204'
 'Alcatel Club Plus Handy']
['Verizon Wireless' 'Phone Arena' 'Amazon' 'Samsung' 'Very' 'CNET'
 'Wireless AT&T' 'Littlewoods' 'Bit.au' 'Smartprix' 'Zap.il'
 'Verkkokauppa' 'Ciao' 'Komplett.no' 'Coolblue' 'ElkjÃ¸p' 'Tkat' 'Wehkamp'
 'CZC' 'KIESKEURIG' 'Elgiganten' 'Wisebuy' 'MPX' 'Phone House' 'Inet'
 'Gigantti' 'Komplett.se ' 'Zoom.br' 'LDLC' 'Walmart' 'Ð¡Ð²Ñ\x8fÐ·Ð½Ð¾Ð¹'
 'Otzyvua' 'Mobi Guru' 'Phone Scoop' 'HSN' 'Optus' 'Kabum' 'Samsung Fun'
 'Fnac' 'Meilleur Mobile' 'Prisjakt' 'Cissa Magazine' 'Yandex' 'Ofertou'
 'Pixmania' 'BalÃ£o da InformÃ¡tica' 'CDON' 'MVideo' 'Irecommend'
 'SmartPhoneShop'

In [37]:
#phone_users.columns
phone_users.nunique() 
g= phone_users['product'].unique()
len(g)# there are 61K unique products.

61314

In [43]:
import re
def strip_character(dataCol):
    r = re.compile(r'[^a-zA-Z !@#$%&*_+-=|\:";<>,./()[\]{}\']')
    return r.sub('', dataCol)


In [50]:
#Check for missing values. Impute the missing values if there is any
phone_users.duplicate

AttributeError: 'DataFrame' object has no attribute 'duplicate'

In [46]:
# round off the score to nearest integer
phone_users['score'] = round(phone_users['score'],0)

In [ ]:
#Check for missing values. Impute the missing values if there is any